<img width="100" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Forest Emissions Tracking - Validation

_CarbonPlan ClimateTrace Team_

This notebook compares our estimates of country-level forest emissions to prior estimates from other
groups. The notebook currently compares againsts:

- Global Forest Watch (Zarin et al. 2016)
- Global Carbon Project (Friedlingstein et al. 2020)


In [ ]:
import geopandas
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt

from carbonplan_styles.mpl import set_theme

set_theme()

In [ ]:
# Input data
# ----------

# country shapes from GADM36
countries = geopandas.read_file("s3://carbonplan-climatetrace/inputs/shapes/countries.shp")

# CarbonPlan's emissions
emissions = pd.read_csv("s3://carbonplan-climatetrace/v0.1/country_rollups.csv")

# GFW emissions
gfw_emissions = pd.read_excel(
    "s3://carbonplan-climatetrace/validation/gfw_global_emissions.xlsx",
    sheet_name="Country co2 emissions",
).dropna(axis=0)
gfw_emissions = gfw_emissions[gfw_emissions["threshold"] == 10]  # select threshold

# Global Carbon Project
gcp_emissions = (
    pd.read_excel(
        "s3://carbonplan-climatetrace/validation/Global_Carbon_Budget_2020v1.0.xlsx",
        sheet_name="Land-Use Change Emissions",
        skiprows=28,
    )
    .dropna(axis=1)
    .set_index("Year")
)
gcp_emissions *= 3.664  # C->CO2
gcp_emissions.index = [pd.to_datetime(f"{y}-01-01") for y in gcp_emissions.index]
gcp_emissions = gcp_emissions[["GCB", "H&N", "BLUE", "OSCAR"]]

In [ ]:
# Merge emissions dataframes with countries GeoDataFrame
gfw_counties = countries.merge(gfw_emissions.rename(columns={"country": "name"}), on="name")
trace_counties = countries.merge(emissions.rename(columns={"iso3_country": "alpha3"}), on="alpha3")

In [ ]:
# reformat to "wide" format (time x country)
trace_wide = (
    emissions.drop(columns=["end_date"])
    .pivot(index="begin_date", columns="iso3_country")
    .droplevel(0, axis=1)
)
trace_wide.index = pd.to_datetime(trace_wide.index)

gfw_wide = gfw_emissions.set_index("country").filter(regex="whrc_aboveground_co2_emissions_Mg_.*").T
gfw_wide.index = [pd.to_datetime(f"{l[-4:]}-01-01") for l in gfw_wide.index]

gfw_wide.head()

## Part 1 - Compare time-averaged country emissions (tropics only)


In [ ]:
# Create a new dataframe with average emissions
avg_emissions = countries.set_index("alpha3")
avg_emissions["trace"] = trace_wide.mean().transpose()

avg_emissions = avg_emissions.set_index("name")
avg_emissions["gfw"] = gfw_wide.mean().transpose() / 1e9

In [ ]:
# Scatter Plot
avg_emissions.plot.scatter("gfw", "trace")
plt.ylabel("Trace [Tg CO2e]")
plt.xlabel("GFW [Tg CO2e]")

## Part 2 - Maps of Tropical Emissions


In [ ]:
avg_emissions_nonan = avg_emissions.dropna()

In [ ]:
kwargs = dict(
    legend=True,
    legend_kwds={"orientation": "horizontal", "label": "Emissions [Tg CO2e]"},
    lw=0.25,
    cmap="Reds",
    vmin=0,
    vmax=1,
)
avg_emissions_nonan.plot("trace", **kwargs)
plt.title("Trace v0")

In [ ]:
avg_emissions_nonan.plot("gfw", **kwargs)

plt.title("GFW Tropics")

In [ ]:
kwargs = dict(
    legend=True,
    legend_kwds={
        "orientation": "horizontal",
        "label": "Emissions Difference [%]",
    },
    lw=0.25,
    cmap="RdBu_r",
    vmin=-40,
    vmax=40,
)
avg_emissions_nonan["pdiff"] = (
    (avg_emissions_nonan["trace"] - avg_emissions_nonan["gfw"]) / avg_emissions_nonan["gfw"]
) * 100
avg_emissions_nonan.plot("pdiff", **kwargs)
plt.title("% difference")

## Part 3 - Compare global emissions timeseries to Global Carbon Project


In [ ]:
ax = gcp_emissions[["H&N", "BLUE", "OSCAR"]].loc["2000":].plot(ls="--")
gcp_emissions["GCB"].loc["2000":].plot(ax=ax, label="GCB", lw=3)
trace_wide.sum(axis=1).plot(ax=ax, label="Trace v0", c="k", lw=3)
plt.ylabel("Emissions [Tg CO2e]")
plt.legend()